## Making tools

### Using technique

In [31]:
## Regular Expression

# s.find('substring') return the indice the substring if exists else -1
# @pattern, origin        ----> pattern 
#           baa!|baaa!    ----> baa*!
# '*' : any numbers of the preceding word
# '?' : zero or one word(exist or not exist)
# '.' : any single character
# '^' : the begging of the text
# '$' : the end of the text

def search(pattern, text):
    "Return True if pattern appears anywhere in text"
    if pattern.startswith('^'):
        return match(pattern[1:] , text)
    else:
#         print('.*' + pattern)
        return match('.*' + pattern, text)
    
def match(pattern, text):
    "return True if pattern appears at the start of text"
    if pattern == '':
        return True
    
    elif pattern == '$':
        return (text == '')
    elif len(pattern) > 1 and pattern[1] in '*?':
        p, op, pat = pattern[0], pattern[1], pattern[2:]
#         print(p, op, pat)
        if op == '*':
            return match_star(p, pat, text)
        elif op == '?':
            if match1(p, text) and match(pat, text):
                return True
            else:
                return match(pat, text)
        
    else:
        return (match1(pattern[0], text) and match(pattern[1:], text[1:]))
    

def match1(p, text):
    """Return True if first character of text natches pattern character p"""
    if not text: return False
    return p == '.' or p == text[0]


def match_star(p, pattern, text):
    """Return true if any number of char p, follwed by pattern, matches text."""
    return (match(pattern, text) or
            (match1(p, text) and match_star(p, pattern, text[1:])))
    
def test():
    #test search(pattern, text)
    assert search('baa*!', 'Sheep said baaaa!') == True
    assert search('baa*!', 'Sheep said baaaa humbu!') == False
    assert search('def', 'abcdefg')
    assert search('def$', 'abcdef')
    assert search('def$', 'abcdefg') == False
    assert search('^start', 'not the start') == False
    assert search('start', 'not the start')
    
    # test match(pattern ,text)
    assert match('baa*!', 'Sheep said baaa!') == False
    assert match('baa*!', 'baaaaaaa! said the sheep')
    assert match('start', 'not the start') == False
    assert match('a*b*c*', 'just anything')
    assert match('x?', 'text')
    assert match('text?', 'text')
    assert match('text?', 'tex')
    
    assert all(match('aa*bb*cc*$', s) 
               for s in words('abc aaabbcc aaaabcccc'))
    assert not any(match('aa*bb*cc*$', s)
                   for s in words('ac aaabbcccd aaaa-b-cccc'))
    assert all(search('^ab.*aca.*a$', s)
              for s in words('abracadavra abacaa about-acacia-a'))
    assert all(search('t.p', s)
              for s in words('tip top tap atypical tepid stop'))
    assert not any(search('t.p', s)
                  for s in words('TYPE teepee tp'))
    
    return 'test passes'
    
def words(text):return text.split()


print(test())

## http://tinyurl.com/pike-regexp

test passes


### Language && Grammer
**language** : a set of strings  
**grammer** : a description of a language

#### Regular Expression

In [ ]:
import re

In [77]:
def search(pattern, text):
    "Match pattern anywhere in text; return longest earliest match or None"
    for i in range(len(text)):
        m = match(pattern, text[i:])
        if m is not None:
            return m
def match(pattern, text):
    "Match pattern againest start of text; return longest match found or None"
    ramainders = matchset(pattern ,text)
    if ramainders:
        shortest = min(remainders, key = len)
        return text[:len(text) - len(shortest)]

def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y

def matchset(pattern, text):
    "Match pattern at start of text; return a set of ramainders of text."
    op, x ,y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]] if text else null)
    elif 'oneof' == op:
        return set([text[1:]]) if any(text.startswith(c) for c in x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                   for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: {}'.format(pattern))
        
null = frozenset()

def lit(string):
#     return ('lit', string)
    return lambda text:set([text[len(string):]]) if text.startswith(string) else null

def seq(x,y):
#     return ('seq', x, y)
    return lambda text:set(t2 for t1 in x(text) for t2 in y(t1))

def alt(x, y):
#     return ('alt', x, y)
    return lambda text: x(text) | y(text)

def star(x):
#     return ('star', x)
    return lambda text:(set([text]) |
                set(t2 for t1 in x(text)
                   for t2 in x(t1) if t1 != text))
def plus(x):
    return ('plus', x)

def opt(x):
    return ('opt', x)

def oneof(chars):
    return ('oneof', tuple(chars))

dot = ('dot',)

eol = ('eol',)

def test_search():
    a,b,c = lit('a'), lit('b'), lit('c')
    abcstars = seq(star(a), star(b), star(c))
    dotstar = star(dot)
    r = seq(lit('ab'), seq(dotstar, seq(lit('aca'), seq(dotstar, seq(a, eol))))) 
    
    assert search(lit('def'), 'abcdefg') == 'def'
    assert search(seq(lit('def'), eol), 'abcdef') == 'def'
    assert search(seq(lit('def'), eol), 'abcdefg') == None
    assert search(a, 'not the start') == 'a'
    
    assert match(a, 'not the start') == None
    assert match(abcstar, 'aaabbbccccccccdef') == 'aaabbbccccccccdef'
    assert match(abcstars, 'junk') == ''
    
    assert all(match(seq(abcstars, eol), s) == s
              for s in 'abc aaabbccc aaaabcccc'.split())
    assert all(match(seq(abcstar, eol), s) == None
              for s in 'cab aaabbcccd aaaa-b-cccc'.split())
    assert all(search(r, s) is not None
              for s in 'abracadabra abacaa about-acacia-flora'.split())
    assert all(match(seq(c, seq(dotstar, b)), s) is not None
              for s in 'cab cab carob ca carbuncle'.split())
    assert not any(match(seq(c, seq(dot,b)), s) 
                  for s in 'crab cb across scab'.split())
    
    return 'test_search passes'

In [59]:
matchset(seq(dot, lit('2')), '123')

{'3'}

In [78]:
pat = seq(lit('a'), lit('b'))
pat('abc')

{'c'}

### compiler && interpreter

In [80]:
import dis
dis.dis(lambda x, y: aqrt(x ** 2 + y ** 2))

  2           0 LOAD_GLOBAL              0 (aqrt)
              2 LOAD_FAST                0 (x)
              4 LOAD_CONST               1 (2)
              6 BINARY_POWER
              8 LOAD_FAST                1 (y)
             10 LOAD_CONST               1 (2)
             12 BINARY_POWER
             14 BINARY_ADD
             16 CALL_FUNCTION            1
             18 RETURN_VALUE
